In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time


In [2]:
def initializer():
    s=Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    time.sleep(5)
    return driver

def load_page(driver,date='20200209'):
    base='https://coinmarketcap.com/historical/'
    driver.get(base+date)
    time.sleep(5)


def scroll_down_bottom_page(driver):
    actions = ActionChains(driver)
    actions.send_keys(Keys.END)
    actions.perform()

def load_more_click(driver):
    try:   
        load_more_xpath='//*[@id="__next"]/div[1]/div[2]/div/div[1]/div[3]/div[2]/button'
        driver.find_element(By.XPATH,load_more_xpath).click()
        return True
    except:
        return False

def load_more_rows(driver):
    scroll_down_bottom_page(driver)
    time.sleep(5)
    response=load_more_click(driver)
    time.sleep(10)
    return response

def scroll_down_page(driver):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
            driver.execute_script("window.scrollTo(0, "+str(current_scroll_position)+")")
            current_scroll_position += 1000  
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(1.5)

def load_all_rows(driver):
    response=True
    while(response):
        response=load_more_rows(driver)

def table_to_dataframe(driver):
    html=driver.page_source
    data=pd.read_html(html)
    df=data[2]
    return df

def date(driver):
    date=(driver.current_url).split('/')[-2]
    return date

def save(driver,path=''):
    df=table_to_dataframe(driver)
    name=date(driver)
    df.iloc[:,1:4].to_csv(path+name+'.csv',index=False)

def next_week_load(driver):
    datetime=date(driver)
    datetime=(pd.to_datetime(datetime)+pd.Timedelta(days=7)).strftime('%Y%m%d')    
    load_page(driver,datetime)
    time.sleep(10)
    



In [3]:
driver=initializer()
load_page(driver,date='20210829')
next_week_load(driver)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Membro\.wdm\drivers\chromedriver\win32\95.0.4638.69]


In [4]:

for i in range(0,100):
    t0=time.clock()
    load_all_rows(driver)
    scroll_down_page(driver)
    save(driver)
    next_week_load(driver)
    t1=time.clock()
    dt=t1-t0
    print(dt)




C:\Users\Membro\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
C:\Users\Membro\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


1544.5493552
1595.9772062000002
1629.6747878
1735.0567726
1765.5326926000007
1923.390319
3141.991559100001
2266.5036843


KeyboardInterrupt: 